Run: docker compose up -d

In [2]:
import os

import pickle
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

In [3]:
METRIC_TYPE = "IndexFlatL2" # Która metoda ma być użyta do porównywania
TEXT_TO_SEARCH = 30         # Numer sentencji do porównania
NN_NUMBER = 5               # Ilość sąsiadów do odszukania

# Lista metod porównujących
METRIC_LIST = {
    "IndexFlatL2": faiss.IndexFlatL2,
    "IndexHNSWFlat": faiss.IndexHNSWFlat,
    "IndexFlatIP": faiss.IndexFlatIP
}

In [4]:
# Tekst – 10 przykładowych zdań
sentences = [
    "Słońce świeci na niebie.", # 0
    "Dzisiaj jest piękna pogoda.",
    "Lubię uczyć się programowania.",
    "Python to popularny język programowania.",
    "Wieczorem często czytam książki.",
    "Mój pies lubi biegać po parku.",
    "Programowanie to świetna zabawa.",
    "Niebo jest dziś bardzo niebieskie.",
    "Czytanie rozwija wyobraźnię.",
    "Chodzę codziennie na spacer z psem.",
    "Lubię Mazury, morze i góry.", # 10
    "Java to mój ulubiony język programowania.",
    "Ostatnio częściej programuję w Pythonie",
    "SQL też nie jest zły.",
    "Za tydzień jedziemy całą ekipą na wakacje w góry.",
    "Zawsze na wyjazdy wakacyjne zabieramy naszego psa.",
    "Dostałem wczoraj mandat za przektoczenie predkości.",
    "Jazda moim Porshe wyzwala we mnie mega emocje.",
    "Kolarstwo górskie daje mi wiele frajdy.",
    "Wczoraj byłem w kinie.",
    "Mam uczulenie na czekoladę ale za to mogę jeść marmoladę.", # 20
    "Najlepsze pączki są z marmoladą i lukrem.",
    "Twój tort urodzinowy był pyszny.",
    "Od roku ograniczam cukier i czuję sie wyśmienicie.",
    "Wczoraj kupiłem poradnik pt. Dieta dla każdego.",
    "Sposób odżywiania ma wpływ na nasze samopoczucie.",
    "Właśnie skończyłem czytać fajną książkę.",
    "Czytanie książek to moja ulubiona zabawa.",
    "Wczoraj byłem w bibliotek i wypożyczyłem kilka książek na temat programowania.",
    "Wczoraj wróciliśmy z wakacji z Mazur.",
    "Mój pies lubi gonić koty." # 30
]

In [ ]:
# Model do embeddingów
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(sentences).astype(np.float32)
vector_dim = embeddings.shape[1]

# Ścieżki do plików
index_file = "index_faiss/sentences_faiss.index"
# sentences_file = "index_faiss/sentences.pkl"

# Tworzenie indeksu FAISS (IndexFlatL2 = najbliżsi sąsiedzi na podstawie odległości euklidesowej)
if not os.path.exists(index_file):
    index = METRIC_LIST[METRIC_TYPE](vector_dim)
    index.add(embeddings)
    faiss.write_index(index, index_file)
    print(f"Indeks FAISS zapisany do: {index_file}")
else:
    print(f"Indeks FAISS już istnieje: {index_file}")

# Wczytanie indeksu i zdań
index = faiss.read_index(index_file)

# Zapytanie
query_id = TEXT_TO_SEARCH
query_vec = embeddings[query_id].reshape(1, -1) # Budowa macierzy 2D

print(f"Zapytanie: {sentences[query_id]}")

# Wyszukiwanie 3 najbliższych sąsiadów
D, I = index.search(query_vec, NN_NUMBER)

# Wyświetlenie wyników
print("Najbardziej podobne zdania:")
for idx, dist in zip(I[0], D[0]):
    print(f"- {sentences[idx]} - (L2 dist: {dist:.4f})")


Indeks FAISS już istnieje: index_faiss/sentences_faiss.index

🔎 Zapytanie: Mój pies lubi gonić koty.

🔗 Najbardziej podobne zdania:
- "Mój pies lubi gonić koty." (L2 dist: 0.0000)
- "Mój pies lubi biegać po parku." (L2 dist: 0.4342)
- "Dzisiaj jest piękna pogoda." (L2 dist: 0.9790)
- "Czytanie książek to moja ulubiona zabawa." (L2 dist: 1.0927)
- "Kolarstwo górskie daje mi wiele frajdy." (L2 dist: 1.1293)
